In [ ]:
# Import necessary packages
import os
import pathlib
import re # Parsing info out of file names

import geopandas as gpd # Working with vector data
import pandas as pd # Aggregating and data manipulation
import rioxarray as rxr # Work with raster data

In [ ]:
# Set up a project data directory

## Define study area:

* Download the USFS National Grassland Units

* Select study sites

In [ ]:
# Define the download URL for the study area

# Check that the URL is correct

In [ ]:
# Try downloading from the URL
# Define a path to save the study area
# Only download once
if not os.path.exists(study_area_path):
    # Download the study area

    # Save the study area

# Load the study area

# Make a plot to check that the data is what we expect

In [ ]:
# Select the study sites

# Make two site maps

## Load in soil data

For each grassland:
    Download at least one soil variable from the POLARIS dataset

In [ ]:
# Define the download URL template for the soil data

# List out all the soil files we will need to download

output_da_list = {}
# Loop through each of the soil files
for ... in soil_files:
    # Download raster data

    # Unzip the raster data

    # Extract metadata from the file name

    # Only download once -- Save raster data to the disk

    # Load the raster data into Python, mask and scale and squeeze w/ .tif files

    # Make sure that the CRS matches between the vector and raster data, this could also be done up above the output_da_list is defined
    #.to_crs(4326)

    # Crop the raster data
    cropped_da = ...

    # Make a plot to check the download

    output_da_list.append(cropped_da)

# Merge tiles



## MORE THINGS NEED TO HAPPEN HERE

In [ ]:
# EXAMPLE of a for loop and accumulating the answers

a_list = [1, 2, 3, 4]

#w/o this accumulator list, when we run the for loop below w/o the prints, we just see 16
squared_number_list = []

for number in a_list:
    #print(number)
    squared_number = number**2
    #print(squared_number)
    squared_number_list.append(squared_number)

print(squared_number)
squared_number_list

16


[1, 4, 9, 16]